In [517]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
import numpy as np

In [518]:
def calculate_probCond(x, y, C, a, V):
    count = 0
    sum_count = 0
    #y.reshape((len(y),1))
    x2 = x[:]
    for i, value_y in enumerate(y):
        if value_y == C:
            count = count + (len(x2[i].split(a)) - 1) # contagem(xk,Cq)
            sum_count = sum_count + len(x[i].split(' ')) # a soma total de palavras pertencentes à classe Cq
    
    probCond = (count + 1) / (sum_count + V)
    return probCond

def calculate_probability(x, y, xTest, print_a_priori=True):
    listNoRepClass = []
    for j in y:
        if j not in listNoRepClass:
            listNoRepClass.append(j) # I have the classes
    
    quantity = []
    count = 0
    total = 0
    for j in listNoRepClass:
        for i in y:
            if i == j: count += 1
        quantity.append(count) # Now in quantity is how many times a class is repeated
        total = total + count
        count = 0
    
    proClass = [j / total for j in quantity] # Probability for each class (priori) 
    
    x1 = x[:]
    listNoRepAtr = []
    for i in x1:
        listx = i.split(' ') 
        for j in listx:
            if j not in listNoRepAtr:
                listNoRepAtr.append(j)
    V = len(listNoRepAtr) # tamanho do vocabulário
    
    probabilityXtest = 1
    Posteriori = []
    for index, j in enumerate(listNoRepClass):
        for i in xTest: 
            probabilityCon = calculate_probCond(x, y, j, i, V)
            probabilityXtest = probabilityXtest*probabilityCon
        Posteriori.append(probabilityXtest*proClass[index])
        probabilityXtest = 1      
    
    for index, j in enumerate(listNoRepClass):
        print('A probabilidade da classe "' + str(j) + '" a posteriori é ' + str(Posteriori[index]))
    if print_a_priori == True:
        print('')
        for index, j in enumerate(listNoRepClass):
            print('A probabilidade da classe "' + str(j) + '" a priori é ' + str(proClass[index]))

In [519]:
X_train = np.array(['Chinese Beijing Chinese', 'Chinese Chinese Shanghai', 'Chinese Macao', 'Tokyo Japan Chinese'])
y_train = np.array(['china','china','china','not china'])
y_1 = np.copy(y_train)
for i in range(0,len(y_train)):
    if y_1[i] == 'china': y_train[i] = 1
    else: y_train[i] = 0

In [520]:
X_test = np.array(['Chinese Chinese Chinese Tokyo Japan'])

## Bernoulli

In [521]:
# Convert a collection of text documents into a matrix of token counts.
vect = CountVectorizer(binary=True)
# Learn the vocabulary dictionary and return term-document matrix.
# This is equivalent to fit followed by transform, but more efficiently implemented.
X_train_dtm = vect.fit_transform(X_train)

In [522]:
# Instantiate a Bernoulli Naive Bayes model.
nb = BernoulliNB(binarize=None)

In [523]:
# Train the BernoulliNB model.
nb.fit(X_train_dtm, y_train)

BernoulliNB(alpha=1.0, binarize=None, class_prior=None, fit_prior=True)

In [524]:
# Transform document into document-term matrix.
X_test_dtm = vect.transform(X_test)
# Perform classification on an array of test vectors X_test_dtm.
y_pred_class = nb.predict(X_test_dtm)
if y_pred_class == '1': y_pred_class = 'china'
else: y_pred_class = 'not china'
    
print('a)\nNome dos atributos:')
print(vect.get_feature_names())
print('\nMatriz de contagem dos atributos para cada uma das instâncias de CountVectorizer')
print(X_train_dtm.toarray())

print('\nb)\nA mensagem "'+  X_test[0] + '" pertence à classe "' + y_pred_class + '".')

a)
Nome dos atributos:
['beijing', 'chinese', 'japan', 'macao', 'shanghai', 'tokyo']

Matriz de contagem dos atributos para cada uma das instâncias de CountVectorizer
[[1 1 0 0 0 0]
 [0 1 0 0 1 0]
 [0 1 0 1 0 0]
 [0 1 1 0 0 1]]

b)
A mensagem "Chinese Chinese Chinese Tokyo Japan" pertence à classe "not china".


## Multinomial

In [525]:
# Convert a collection of text documents to a matrix of token counts.
cv = CountVectorizer()
# Naive Bayes classifier for multinomial models.
mnb = MultinomialNB()
# Create a pipeline that attaches the vectorizer to a multinomial naive Bayes classifier.
model = make_pipeline(cv, mnb)

In [526]:
# Train model. Apply the model to the training data.
model.fit(X_train, y_train)
# Run validation. Predict labels for the test data.
y_pred_class = model.predict(X_test)
if y_pred_class == '1': y_pred_class = 'china'
else: y_pred_class = 'not china'

X_train_dtm = cv.fit_transform(X_train)
print('a)\nNome dos atributos:')
print(vect.get_feature_names())
print('\nMatriz de contagem dos atributos para cada uma das instâncias de CountVectorizer')
print(X_train_dtm.toarray())

print('\nb)\nA mensagem "'+  X_test[0] + '" pertence à classe "' + y_pred_class + '".')

a)
Nome dos atributos:
['beijing', 'chinese', 'japan', 'macao', 'shanghai', 'tokyo']

Matriz de contagem dos atributos para cada uma das instâncias de CountVectorizer
[[1 2 0 0 0 0]
 [0 2 0 0 1 0]
 [0 1 0 1 0 0]
 [0 1 1 0 0 1]]

b)
A mensagem "Chinese Chinese Chinese Tokyo Japan" pertence à classe "china".


In [527]:
print('c)')
X_test1 = X_test[0].split(' ')
probability = calculate_probability(X_train, y_1, X_test1)

c)
A probabilidade da classe "china" a posteriori é 0.00030121377997263036
A probabilidade da classe "not china" a posteriori é 0.00013548070246744226

A probabilidade da classe "china" a priori é 0.75
A probabilidade da classe "not china" a priori é 0.25


In [528]:
#probability = calculate_probability(X_test, y_pred_class)
print('d)')
print('Nome dos atributos:')
print(vect.get_feature_names())
print('\nVetor de contagens de cada classificador em Bernoulli:')
print(X_test_dtm.toarray())
X_test_dtm1 = cv.transform(X_test)
print('\nVetor de contagens de cada classificador em Multinomial:')
print(X_test_dtm1.toarray())

d)
Nome dos atributos:
['beijing', 'chinese', 'japan', 'macao', 'shanghai', 'tokyo']

Vetor de contagens de cada classificador em Bernoulli:
[[0 1 1 0 0 1]]

Vetor de contagens de cada classificador em Multinomial:
[[0 3 1 0 0 1]]


In [529]:
print('No caso do Bernoulli o vetor de contagens somente apresenta se existe a palavra ou não, isso faz com que ele interprete uma ocurrência para a clase "china " e duas ocurrências para a classe "not china". Como têm maior quantidade de ocurrências para a classe "not china", o modelo treinado com Bernoulli prediz erroneamente que a mensagem pertence à classe "not china".\n')
print('No caso do Multinomial o vetor de contagens apresenta se existe a palavra e as vezes que ela se repite, isso faz como que ele interprete três ocurrências para a classe "china" e duas ocurrências para a classe "not china". Como têm maior quantidade de ocurrências para a classe "china", o modelo treinado com Multinomial prediz corretamente que a mensagem pertence à classe "china".\n')
print("Então, em Bernoulli, é como se a mensagem (X_test1) fosse ['Chinese', 'Tokyo', 'Japan'] y em Multinomial fosse ['Chinese Chinese Chinese Tokyo Japan']. Agora vou a calcular manualmente a probabilidade de cada classe, ou seja, ‘china’ e ‘not china’, para cada mensagem de teste para os 2 classificadores.")

print('\nProbabilidade de cada classificador em Bernoulli:')
X_test1 = ['Chinese', 'Tokyo', 'Japan']
probability1 = calculate_probability(X_train, y_1, X_test1, print_a_priori=False)

print('\nProbabilidade de cada classificador em Multinomial:')
X_test1 = ['Chinese', 'Chinese', 'Chinese', 'Tokyo', 'Japan']
X_test1 = X_test[0].split(' ')
probability = calculate_probability(X_train, y_1, X_test1, print_a_priori=False)

No caso do Bernoulli o vetor de contagens somente apresenta se existe a palavra ou não, isso faz com que ele interprete uma ocurrência para a clase "china " e duas ocurrências para a classe "not china". Como têm maior quantidade de ocurrências para a classe "not china", o modelo treinado com Bernoulli prediz erroneamente que a mensagem pertence à classe "not china".

No caso do Multinomial o vetor de contagens apresenta se existe a palavra e as vezes que ela se repite, isso faz como que ele interprete três ocurrências para a classe "china" e duas ocurrências para a classe "not china". Como têm maior quantidade de ocurrências para a classe "china", o modelo treinado com Multinomial prediz corretamente que a mensagem pertence à classe "china".

Então, em Bernoulli, é como se a mensagem (X_test1) fosse ['Chinese', 'Tokyo', 'Japan'] y em Multinomial fosse ['Chinese Chinese Chinese Tokyo Japan']. Agora vou a calcular manualmente a probabilidade de cada classe, ou seja, ‘china’ e ‘not china’

In [530]:
print('Pode-se verificar que a probabilidade a posteriori no classificador do Bernoulli é maior para a classe "not china" e em Multinomial é maior para a classe "china", do mesmo jeito que o metodo implementado pela libreria sklearn.')

Pode-se verificar que a probabilidade a posteriori no classificador do Bernoulli é maior para a classe "not china" e em Multinomial é maior para a classe "china", do mesmo jeito que o metodo implementado pela libreria sklearn.
